<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_text_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

In [2]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (85/85), 993.54 KiB | 18.06 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/cloned-repo
dataset.py  dsiplay_ground_truth.ipynb	EAST_text_detector.ipynb  model.py
detect.py   EAST_detection.ipynb	loss.py			  train.py
CPU times: user 75.1 ms, sys: 16 ms, total: 91 ms
Wall time: 4.37 s


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from dataset import *

In [5]:
# AI4Bharat dataset
image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Images'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Annotations'


# icdar dataset
# image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Images'
# label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Labels'

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

428


## Dataset Initial Check

In [6]:
img = Image.open(image_address+'/5.jpg')

with open(label_address+'/5.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[ 69.78117866 462.90128661 333.08121585 459.57105882 318.45199036
 558.10017006  70.80303846 543.99355383]
--------------------
[352.95000447 436.28519962 652.14476548 409.25752889 656.48100592
 563.52093031 342.61946157 553.98515665]
--------------------
[1 1]


In [7]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(695.)

## Loss for one batch

In [8]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=8)

imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))

print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())

print(torch.sum(gt_score))

torch.Size([8, 3, 512, 512]) torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128]) torch.Size([8, 1, 128, 128])
tensor(8733.)
CPU times: user 5.03 s, sys: 4.64 s, total: 9.67 s
Wall time: 10.1 s


In [9]:
from model import EAST

model = EAST()

Downloading: "http://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:11<00:00, 46.9MB/s]


In [10]:
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128])


In [11]:
from loss import Loss

loss_fn = Loss()

loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.86176562, angle loss is 0.74476987, iou loss is 2.91190028


tensor(11.2214, grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
from model import EAST

model = EAST()
state_dict = torch.load('/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/model_epoch_20.pth')
model.load_state_dict(state_dict)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[epoch_iter//2], gamma=0.1)
  
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))

In [14]:
train(model, image_address, label_address, batch_size=8, lr=1e-4, epoch_iter=30)

428


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.31213045, angle loss is 0.00794076, iou loss is 0.67967618
Epoch is [1/30], mini-batch is [1/53], time consumption is 1.00603032, batch_loss is 1.07121420
classify loss is 0.23912942, angle loss is 0.00551597, iou loss is 0.65037882
Epoch is [1/30], mini-batch is [2/53], time consumption is 0.95631003, batch_loss is 0.94466794
classify loss is 0.23795074, angle loss is 0.00457440, iou loss is 0.72712189
Epoch is [1/30], mini-batch is [3/53], time consumption is 0.86457944, batch_loss is 1.01081657
classify loss is 0.27450645, angle loss is 0.00798017, iou loss is 0.65521944
Epoch is [1/30], mini-batch is [4/53], time consumption is 0.77727485, batch_loss is 1.00952756
classify loss is 0.27236134, angle loss is 0.00937838, iou loss is 0.71118224
Epoch is [1/30], mini-batch is [5/53], time consumption is 0.93993163, batch_loss is 1.07732737
classify loss is 0.29257357, angle loss is 0.00537087, iou loss is 0.63485056
Epoch is [1/30], mini-batch is [6/53], time consumpt